# BEE 4750 Homework 3: Dissolved Oxygen and Monte Carlo

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 10/03/23, 9:00pm

## Overview

### Instructions

-   Problem 1 asks you to implement a model for dissolved oxygen in a
    river with multiple waste releases and use this to develop a
    strategy to ensure regulatory compliance.
-   Problem 2 asks you to use Monte Carlo simulation to assess how well
    your strategy from Problem 1 performs under uncertainty.
-   Problem 3 (5750 only) asks you to identify where a third discharge
    should be placed to maintain regulatory compliance.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/Desktop/Julia Scripts/hw3-team-puffin`
   Installed SentinelArrays ────────── v1.4.5
   Installed InlineStrings ─────────── v1.4.2
   Installed StatsFuns ─────────────── v1.3.1
   Installed Pango_jll ─────────────── v1.52.2+0
   Installed Rmath_jll ─────────────── v0.4.2+0
   Installed HypergeometricFunctions ─ v0.3.23
   Installed PtrArrays ─────────────── v1.2.0
   Installed TranscodingStreams ────── v0.11.1
   Installed Rmath ─────────────────── v0.7.1
   Installed Tables ────────────────── v1.12.0
   Installed QuadGK ────────────────── v2.9.4
   Installed FillArrays ────────────── v1.11.0
   Installed Distributions ─────────── v0.25.109
Precompiling project...
  ✓ Calculus
  ✓ SentinelArrays
  ✓ FillArrays
  ✓ InlineStrings
  ✓ PtrArrays
  ✓ TranscodingStreams
  ✓ PooledArrays
  ✓ Tables
  ✓ FilePathsBase
  ✓ Rmath_jll
  ✓ OpenSpecFun_jll
  ✓ StringManipulation
  ✓ Dbus_jll
  ✓ QuadGK
  ✓ Pango_jll
  ✓ InlineStrings → ParsersExt
  ✓ AliasTables
  ✓ Transc

In [75]:
using Random
using Plots
using LaTeXStrings
using Distributions
using CSV
using DataFrames

## Problems (Total: 50/60 Points)

### Problem 1 (30 points)

A river which flows at 6 km/d is receiving waste discharges from two
sources which are 15 km apart. The oxygen reaeration rate is 0.55
day<sup>-1</sup>, and the decay rates of CBOD and NBOD are are 0.35 and
0.25 day<sup>-1</sup>, respectively. The river’s saturated dissolved
oxygen concentration is 10m g/L.

If the characteristics of the river inflow and waste discharges are
given in <a href="#tbl-river" class="quarto-xref">Table 1</a>, write a
Julia model to compute the dissolved oxygen concentration from the first
wastewater discharge to an arbitrary distance `d` km downstream. Use
your model to compute the maximum dissolved oxygen concentration up to
50km downstream and how far downriver this maximum occurs.

| Parameter | River Inflow | Waste Stream 1 | Waste Stream 2 |
|:--:|---:|---:|---:|
| Inflow | 100,000 m<sup>3</sup>/d | 10,000 m<sup>3</sup>/d | 15,000 m<sup>3</sup>/d |
| DO Concentration | 7.5 mg/L | 5 mg/L | 5 mg/L |
| CBOD | 5 mg/L | 50 mg/L | 45 mg/L |
| NBOD | 5 mg/L | 35 mg/L | 35 mg/L |

Table 1: River inflow and waste stream characteristics for Problem 1.

**In this problem**:

-   Plot the dissolved oxygen concentration from the first waste stream
    to 50m downriver. What is the minimum value in mg/L?
-   What is the minimum level of treatment (% removal of organic waste)
    for waste stream 1 that will ensure that the dissolved oxygen
    concentration never drops below 4 mg/L, assuming that waste stream 2
    remains untreated? How about if only waste stream 2 is treated?
-   Suppose you are responsible for designing a waste treatment plan for
    discharges into the river, with a regulatory mandate to keep the
    dissolved oxygen concentration above 4 mg/L. Discuss whether you’d
    opt to treat waste stream 2 alone or both waste streams equally.
    What other information might you need to make a conclusion, if any?

In [163]:
function initial_conditions(Xr, Qr, X1, Q1)     #Calculates initial conditions based on initial and additional flows
    numerator = (Xr * Qr) + (X1 * Q1)   #Total concentration of desired metric of combined flows
    flow_total = Qr + Q1                #Calculates total volume of combined flows
    Xa = (numerator / flow_total)       #Calculates resulting metric of combined flow
end

function do_remaining(x, C0, B0, N0, ka, kc, kn, Cs, U)
    B = B0 * exp(-kc * x / U)
    N = N0 * exp(-kn * x / U)
    a1 = exp(-ka * x / U)
    a2 = (kc / (ka-kc)) * (exp(-kc * x / U) - exp(-ka * x / U))
    a3 = (kn / (ka-kn)) * (exp(-kn * x / U) - exp(-ka * x / U))
    C = (Cs * (1 - a1) + (C0 * a1) - (B * a2) - (N * a3))
    return (C, B, N)
end

ka = 0.55           #reaeration rate (day^-1)
kc = 0.35           #CBOD uptake rate (day^-1)
kn = 0.25           #NBOD uptake rate (day^-1)
U = 6               #River speed (km/day)
Cs = 10 * 10^3      #Fully saturated concentration (mg/m^3)

Qr = 100000         #River inflow volume (m^3/day)
Cr = 7.5 * 10^3     #River DO concentration (mg/m^3)
Br = 5 * 10^3       #River CBOD
Nr = 5 * 10^3       #River NBOD

Q1 = 10000          #Source 1 Inflow volume
C1 = 5 * 10^3       #Source 1 DO concentration
B1 = 50 * 10^3      #Source 1 CBOD
N1 = 35 * 10^3      #Source 1 NBOD

Q2 = 15000          #Source 2 Inflow Volume
C2 = 5 * 10^3       #Source 2 DO concentration
B2 = 45 * 10^3      #Source 2 CBOD
N2 = 35 * 10^3      #Source 2 NBOD

x1 = 0:15           #Distance with inputs from River and Source 1
x2 = 0:35          #Distance with inputs from River, Source 1, Source 2

Ca = initial_conditions(Cr, Qr, C1, Q1)     #DO Concentration where River merges with Source 1
Ba = initial_conditions(Br, Qr, B1, Q1)     #CBOD where River merges with Source 1
Na = initial_conditions(Nr, Qr, N1, Q1)     #NBOD where River merges with Source 1
Qa = Q1 + Qr                                #Total Flow volume after merge with Source 1

do_out = (y -> do_remaining(y, Ca, Ba, Na, ka, kc, kn, Cs, U)).(x1)
C = [d[1] for d in do_out]           
B = [d[2] for d in do_out]
N = [d[3] for d in do_out]

Cb = initial_conditions(C[15], Qa, C2, Q2)
Bb = initial_conditions(B[15], Qa, B2, Q2)
Nb = initial_conditions(N[15], Qa, N2, Q2)

do_out2 =  (y -> do_remaining(y, Cb, Bb, Nb, ka, kc, kn, Cs, U)).(x2)
append!(C, d[1] for d in do_out2)
deleteat!(C, 16)
append!(B, d[2] for d in do_out2)
deleteat!(B, 16)
append!(N, d[3] for d in do_out2)
deleteat!(N, 16)

C /= 1000
B /= 1000
N /= 1000
xCoords = 0:50

plot(xCoords, C, xlabel="Distance from Source 1 (km)", ylabel="Dissolved Oxygen (mg/L)")
plot!(title="Dissolved Oxygen Concentration along the River")
plot!(xCoords, B)
plot!(xCoords, N)



In [164]:
minDO = minimum(C)

5.682585825002739

In [ ]:
#Due to some unknown error in my calculations, the drop in DO (and rise in CBOD and NBOD) expected at
#15 km (when the sencond inflow merges with the river) is as high as expected.

### Problem 2 (20 points)

The simplest climate model involves capturing changes to the Earth’s
energy budget (it is commonly called the *energy balance model*, or
EBM). These changes are also called *radiative forcings* (RF), and can
result from several causes, including greenhouse gas emissions, volcanic
eruptions, and changes to the solar cycle. The EBM treats the Earth as a
0-dimensional sphere covered with water, which absorbs heat in response
to radiative forcings. Chanwith global temperature changes resulting
from imbalances in the average (over the entire surface area) heat flux.

The EBM equations are:

$$
\begin{align*}
\overbrace{\frac{dH}{dt}}^{\text{change in heat}} &= \overbrace{F}^{\substack{\text{radiative} \\ \text{forcing}}} - \overbrace{\lambda T}^{\substack{\text{change in} \\ \text{temperature}}} \\
\underbrace{C}_{\substack{\text{ocean heat} \\ \text{capacity}}} \frac{dT}{dt} &= F - \lambda T \\
c\underbrace{d}_{\substack{\text{ocean} \\ \text{mixing depth}}} \frac{dT}{dt} &= F - \lambda T,
\end{align*}
$$

where $c = 4.184\times 10^6 \mathrm{J/K/m}^2$ is the specific heat of
water per area, $d$ is the depth of the ocean mixed layer (we’ll assume
$d = 86 \mathrm{m}$), and $\lambda$ is the **climate feedback factor**
and controls how much the Earth warms in response to increased radiative
forcing (assume
$\lambda = 2.1^\circ \mathrm{C}/(\mathrm{W}/\mathrm{m}^2$)). The total
radiative forcing $F = F_\text{non-aerosol} + \alpha F_\text{aerosol}$,
where $\alpha$ is an uncertain scaling factor reflecting aerosol-cloud
feedbacks (we’ll assume $\alpha = 0.8$).

The code below loads historical and projected radiative forcings (under
the SSP5-8.5 future emissions scenario, which is the most extreme of the
scenarios used to project climate change impacts) from
`data/ERF_ssp585_1750-2500.csv` into a `DataFrame` object and calculates
the non-aerosol and aerosol components of those forcings.

> **Tip**
>
> Look closely at and experiment with the code below: `DataFrames` are a
> common Julia datatype for tabular data, and you may work more with
> them later in the semester or beyond! They are broadly similar to
> `DataFrames` from `pandas` in Python.

In [71]:
# Dataset from https://zenodo.org/record/3973015
# The CSV is read into a DataFrame object, and we specify that it is comma delimited
forcings_all = CSV.read("data/ERF_ssp585_1750-2500.csv", DataFrame, delim=",")

# Separate out the individual components
# Get total aerosol forcings
forcing_aerosol_rad = forcings_all[!,"aerosol-radiation_interactions"]
forcing_aerosol_cloud = forcings_all[!,"aerosol-cloud_interactions"]
forcing_aerosol = forcing_aerosol_rad + forcing_aerosol_cloud
# Calculate non-aerosol forcings from the total.
forcing_total = forcings_all[!,"total"]
forcing_non_aerosol = forcing_total - forcing_aerosol


751

We can plot the aerosol and non-aerosol forcings below.

In [53]:
t = Int64.(forcings_all[!,"year"]) # Ensure that years are interpreted as integers
p_forcing = plot(; xlabel="Year", ylabel="Radiative Forcing (W/m²)")
plot!(p_forcing, t, forcing_aerosol, label="Aerosol Forcing", color=:blue, linewidth=2)
plot!(p_forcing, t, forcing_non_aerosol, label="Non-Aerosol Forcing", color=:red, linewidth=2)

GKS: dlopen(/Users/rharwell/.julia/artifacts/5bbe7f3fc3d33302eca426fc365cd42f54e267a7/lib/cairoplugin.so, 1): Library not loaded: @rpath/libcairo.2.dylib
  Referenced from: /Users/rharwell/.julia/artifacts/5bbe7f3fc3d33302eca426fc365cd42f54e267a7/lib/cairoplugin.so
  Reason: Incompatible library version: cairoplugin.so requires version 11603.0.0 or later, but libcairo.2.dylib provides version 2.0.0


**In this problem**:

-   Discretize the EBM to produce a simulation model of global mean
    temperatures $T$ over time as a result of total radiative forcings
    $F$.

-   Simulate global mean temperature anomalies (in $^\circ C$ relative
    to 1750) from your model using the historical and SSP5-8.5 radiative
    forcing data. Use an annual time step for the simulation (in
    seconds: $\Delta t = 31,558,152 \mathrm{s}$). You can assume
    $T(0) = 0^\circ C$. Plot the resulting temperature simulation.

-   The climate feedback factor $\lambda$ is one of the key
    uncertainties in projecting future temperatures, even assuming a
    particular scenario of future radiative forcing. Suppose we use the
    following distribution for $\lambda$,
    $$\lambda \sim \text{LogNormal}(\log(2.1), \log(2)/4).$$

    Use Monte Carlo simulation to estimate the expected temperature in
    2100 assuming SSP5-RCP 8.5 radiative forcings (with 95% confidence
    intervals). How does this estimate compare to the value you got from
    your simulation using the expected value of
    $\lambda=2.1^\circ \mathrm{C}/(\mathrm{W}/\mathrm{m}^2)$? How did
    you decide your sample set was sufficient?

In [80]:
function tempChange(T0, h, c, d, Fna, Fa, a)
    radForcing = Fna + a * Fa
    tempLoss = h * T0
    TChange = (radForcing - tempLoss) / (c * d)
end

c = 4.184 * 10^6
d = 86
h = 2.1
a = 0.8

Temps = zeros(751)
series_length = length(Temps) - 1

for i = 1:series_length
    Temps[i+1] = Temps[i] + 31558152 * tempChange(Temps[i], h, c, d, forcing_non_aerosol[i], forcing_aerosol[i], a)
end

plot(Temps, xlabel = "Years Since 1750", ylabel = "Temperature deviation from 1750 (˚C)")
plot!(title="Global Mean Temperature Anomalies from 1750 Baseline")

In [92]:
feedback_Dist = LogNormal(log(2.1), log(2)/4)
hrand = rand(feedback_Dist, 750)

TempsRand = zeros(751)
for i = 1:series_length
    TempsRand[i+1] = TempsRand[i] + 31558152 * tempChange(TempsRand[i], hrand[i], c, d, forcing_non_aerosol[i], forcing_aerosol[i], a)
end

plot(TempsRand, xlabel="Years Since 1750", ylabel="Temperature Deviation from 1750 (˚C)")
plot!(title = "Global Mean Temperature Anomalies from 1750 Baseline")


In [165]:
index2100 = 2100 - 1750 + 1
Est2100 = Temps[index2100]

TempsSample = zeros(351)
sampsize = 10
tempSum = 0
hrand2 = zeros(350)
tempSum = zeros(sampsize)

for i = 1:sampsize
    hrand2 = rand(feedback_Dist, 350)
    TempsSample[i+1] = TempsSample[i] + 31558152 * tempChange(TempsSample[i], hrand2[i], c, d, forcing_non_aerosol[i], forcing_aerosol[i], a)
    tempSum[i] = TempsSample[351]
end


Est2100Rand = mean(tempSum)
return Est2100, Est2100Rand

(4.632496731623587, 0.0)

### Problem 3 (10 points)

**This problem is only required for students in BEE 5750**.

A factory is planning a third wastewater discharge into the river
downstream of the second plant. This discharge would consist of 5
m<sup>3</sup>/day of wastewater with a dissolved oxygen content of 4.5
mg/L and CBOD and NBOD levels of 50 and 45 mg/L, respectively.

**In this problem**:

-   Assume that the treatment plan you identified in Problem 1 is still
    in place for the existing discharges. If the third discharge will
    not be treated, under the original inflow conditions (7.5 mg/L DO),
    how far downstream from the second discharge does this third
    discharge need to be placed to keep the river concentration from
    dropping below 4 mg/L?

## References

List any external references consulted, including classmates.